<a href="https://colab.research.google.com/github/ypcio/GenAI_TMLC/blob/main/Finetune_OpenAI_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Install the required library

Make sure you have the `openai` library installed. Use the command below if not already installed:

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
# Import necessary libraries
import os
import pandas as pd
import json
from openai import OpenAI

Access data from this [link](https://themlco-my.sharepoint.com/:f:/p/chiragchauhan/Eg7biBxP1hhEta6JLchrfWgBvLawQsfxSDJO6K1BQl_J9g?e=FhiWT3)

## Step 2: Set up your API key
Replace 'YOUR_API_KEY' with your actual OpenAI API key.

In [ ]:
from google.colab import userdata # check the key shaped the symbol on the left pane of the notebook
OpenAI_API = userdata.get('OpenAI')

In [ ]:
os.environ['OPENAI_API_KEY'] = OpenAI_API

## Step 3: Prepare the dataset

Hint: Use a JSON or CSV file and convert it to JSONL

If loading a CSV dataset and convert it to JSONL format.
Complete the conversion code below:

Here for example directly using the training.jsonl file used during finetuning via playground

In [ ]:
# df = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with your dataset file

# # Convert the dataset to JSONL format
# output_file = 'data.jsonl'
# with open(output_file, 'w') as f:
#     for _, row in df.iterrows():
#         # Create JSON lines for chat model fine-tuning
#         json_line = json.dumps({
#             "messages": [
#                 {"role": "system", "content": "You are a helpful assistant."},
#                 {"role": "user", "content": row['question']},
#                 {"role": "assistant", "content": row['answer']}
#             ]
#         })
#         f.write(json_line + '\n')
# print(f"Dataset converted and saved to {output_file}")

# comment when using your own data
output_file = 'training.jsonl'

## Step 4: Upload the file for fine-tuning

Use the OpenAI API to upload the dataset. Replace '<JSONL_FILE>' with your dataset file name.

In [ ]:
client = OpenAI()

In [ ]:
uploaded_file = client.files.create(
    file=open(output_file, "rb"),
    purpose="fine-tune"
)
print(f"File uploaded successfully. File ID: {uploaded_file.id}")

File uploaded successfully. File ID: file-H2eGyPY7TN8qbKKE1YCgnE


## Step 5: Fine-tune the model

Trigger the fine-tuning job process using the uploaded file ID. Replace 'FILE_ID' and 'MODEL_NAME' accordingly.

In [ ]:
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    suffix="custom-fine-tuned-model",
    model="gpt-4o-mini-2024-07-18"  # Adjust the model as required
)
print(f"Fine-tuning job started. Job ID: {fine_tune_job.id}")

Fine-tuning job started. Job ID: ftjob-mv6DDWyq2YiWf1ECwWvUGqeN


## Step 6: Monitor and use the fine-tuned model

Check list of fine-tuning jobs, retrieve job details.

In [ ]:
# List fine-tuning jobs
jobs = client.fine_tuning.jobs.list(limit=10)
print("Recent fine-tuning jobs:", jobs)

Recent fine-tuning jobs: SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-mv6DDWyq2YiWf1ECwWvUGqeN', created_at=1733475531, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-FZKOY5vDxblsWFjjm6INeCk3', result_files=[], seed=1129603953, status='running', trained_tokens=None, training_file='file-H2eGyPY7TN8qbKKE1YCgnE', validation_file=None, estimated_finish=1733475769, integrations=[], user_provided_suffix='custom-fine-tuned-model'), FineTuningJob(id='ftjob-bKiVUjD3FoyftB7coZbkmj81', created_at=1733475433, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-FZKOY5vDxbl

In [ ]:
# Retrieve job details
job_details = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
print("Fine-tuning job details:", job_details)

Fine-tuning job details: FineTuningJob(id='ftjob-mv6DDWyq2YiWf1ECwWvUGqeN', created_at=1733475531, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-FZKOY5vDxblsWFjjm6INeCk3', result_files=[], seed=1129603953, status='running', trained_tokens=None, training_file='file-H2eGyPY7TN8qbKKE1YCgnE', validation_file=None, estimated_finish=1733475771, integrations=[], user_provided_suffix='custom-fine-tuned-model')


In [ ]:
# Retrieve after model is trained to get the model name
job_details = client.fine_tuning.jobs.retrieve(fine_tune_job.id)
print("Fine-tuning job details:", job_details)

Fine-tuning job details: FineTuningJob(id='ftjob-mv6DDWyq2YiWf1ECwWvUGqeN', created_at=1733475531, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:custom-fine-tuned-model:AbOWr1n9', finished_at=1733475787, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-FZKOY5vDxblsWFjjm6INeCk3', result_files=['file-3ynQi4zaj7y3eV6jxt3Lvm'], seed=1129603953, status='succeeded', trained_tokens=6078, training_file='file-H2eGyPY7TN8qbKKE1YCgnE', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='custom-fine-tuned-model')


## Step 7: Use the fine-tuned model

In [ ]:
# Example call to the fine-tuned model
completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal:custom-fine-tuned-model:AbOWr1n9",
    # Replace with the actual model name retrieved in above cell fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:custom-fine-tuned-model:AbOWr1n9'
    messages=[
        {"role": "system", "content": "You are a helpful assistant which acts as FAQ Support Assistant for the TMLC Guided Projects in Generative AI Program and answer to user queries."},
        {"role": "user", "content": "How long is the program?"}
    ]
)
print("Fine-tuned model response:", completion.choices[0].message.content)

Fine-tuned model response: This is a 6 weeks long program.
